# Import

In [ ]:
#!pip install tensorflow_text
!pip install tensorflow_addons

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text
import tensorflow_addons as tfa
import numpy as np

# Feature Net

In [ ]:
class FeatureNet(tf.keras.Model):
  def __init__(self, K, preprocessor, encoder, isTraining, **kwargs):
    super(FeatureNet, self).__init__(**kwargs)
    self.K = K
    self.preprocessor = preprocessor
    self.encoder = encoder
    self.isTraining = isTraining
    self.bert = BERT(self.preprocessor, self.encoder, self.isTraining)
    self.cbhg = CBHG(self.K, self.isTraining)

  def call(self, inputs):
    outputs = self.bert(inputs)
    outputs = self.cbhg(outputs)
    return outputs


## BERT

In [ ]:
class BERT(tf.keras.Model):
  def __init__(self, preprocessor, encoder, isTraining, **kwargs):
    super(BERT, self).__init__(**kwargs)
    self.preprocessor = preprocessor
    self.encoder = encoder
    self.isTraining = isTraining
    self.preprocess = hub.KerasLayer(preprocessor)
    self.encode = hub.KerasLayer(encoder, trainable=self.isTraining)

  def call(self, inputs):
    outputs = self.preprocess(inputs)
    outputs = self.encode(outputs)
    outputs = tf.expand_dims(outputs["pooled_output"], axis=-1)
    return outputs


## CBHG module

In [ ]:
class Conv1DBank(tf.keras.Model):
  def __init__(self, channels, kernelSize, activation, isTraining, **kwargs):
    super(Conv1DBank, self).__init__(**kwargs)
    self.channels = channels
    self.kernelSize = kernelSize
    self.activation = activation
    self.isTraining = isTraining

    self.conv1d = tf.keras.layers.Conv1D(
        filters=self.channels, kernel_size=self.kernelSize, activation=self.activation, padding='same')
    self.batchNorm = tf.keras.layers.BatchNormalization(trainable=self.isTraining)

  def call(self, inputs):
    outputs = self.conv1d(inputs)
    outputs = self.batchNorm(outputs)
    return outputs
  

In [ ]:
class CBHG(tf.keras.Model):
  def __init__(self, K, isTraining, **kwargs):
    super(CBHG, self).__init__(**kwargs)
    self.K = K
    self.isTraining = isTraining
    self.ConvBanks = [Conv1DBank(128, i, tf.nn.relu, self.isTraining) for i in range(1, self.K + 1)]
    self.maxPooling = tf.keras.layers.MaxPool1D(pool_size=2, strides=1, padding='same')
    self.firstProjectionConv = Conv1DBank(128, 3, tf.nn.relu, self.isTraining)
    self.secondProjectionConv = Conv1DBank(128, 3, None, self.isTraining)
    self.highwayNet = tf.keras.Sequential([tf.keras.layers.Dense(128, tf.nn.relu) for i in range(4)])
    self.bidirectionalGRU = tf.keras.layers.Bidirectional(
        tf.keras.layers.GRU(64, return_sequences=True),
        backward_layer=tf.keras.layers.GRU(64, return_sequences=True, go_backwards=True),
    )
    self.encoderPreNet = tf.keras.Sequential([
        tf.keras.layers.Dense(256, tf.nn.relu),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, tf.nn.relu),
        tf.keras.layers.Dropout(0.5)
    ])
    self.lastProjectionConv = Conv1DBank(1, 3, None, self.isTraining)
    
  def call(self, inputs):
    outputList = []
    for convBank in self.ConvBanks:
      outputList.append(convBank(inputs))
    outputs = tf.keras.layers.concatenate(outputList)
    outputs = self.maxPooling(outputs)
    outputs = self.firstProjectionConv(outputs)
    outputs = self.secondProjectionConv(outputs)
    highwayOutputs = outputs + inputs
    outputs = self.highwayNet(highwayOutputs)
    outputs = self.bidirectionalGRU(outputs)
    outputs = self.encoderPreNet(outputs)
    outputs = self.lastProjectionConv(outputs)
    return outputs


# Test feature net

In [ ]:
PREPROCESSOR = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
ENCODER = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-256_A-4/1"
text_input = ['This is such an amazing movie!', 'This is such an amazing movie!']

In [ ]:
featureNet = FeatureNet(16, PREPROCESSOR, ENCODER, False)
output = featureNet(text_input)

# Utils

## Orthogonal regularizer

In [5]:
class OrthogonalRegularizer(tf.keras.regularizers.Regularizer):
    def __init__(self, beta=1e-4, **kwargs):
        super(OrthogonalRegularizer, self).__init__(**kwargs)
        self.beta = beta

    def call(self, input_tensor):
        c = input_tensor.shape[-1]
        x = tf.reshape(input_tensor, (-1, c))
        ortho_loss = tf.matmul(x, x, transpose_a=True) * (1 - tf.eye(c))
        outputs = self.beta * tf.norm(ortho_loss)
        return outputs

# Generatore



## Generator block

In [25]:
# batch >= 512 -> si rompe tutto
class GeneratorBlock(tf.keras.Model):
  def __init__(self, channels, isTraining, upsampleFactor=1, **kwargs):
    super(GeneratorBlock, self).__init__(**kwargs)
    self.channels = channels
    self.upsampleFactor = upsampleFactor
    self.isTraining = isTraining
    self.stride = 1
    if self.upsampleFactor != 1: self.stride = self.upsampleFactor // 2 
    self.firstCBN = ConditionalBatchNorm(self.channels, self.isTraining)
    self.firstStack = tf.keras.Sequential(
        [
          tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1DTranspose(
                    self.channels, self.upsampleFactor, strides=self.stride, padding='same',
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer())),
          tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1D(channels, 3, padding='same',
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer()))
    ]) # (batch_size, time_steps, self.channels)
    self.secondCBN = ConditionalBatchNorm(self.channels, self.isTraining)
    self.firstDilatedConv =  tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1D(channels, 3, padding='same', dilation_rate=2,
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer()))
    self.residualStack = tf.keras.Sequential(
        [
          tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1DTranspose(
                    self.channels, self.upsampleFactor, strides=self.stride, padding='same',
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer())),
          tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1D(channels, 1, padding='same',
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer()))
    ])
    self.thirdCBN = ConditionalBatchNorm(self.channels, self.isTraining)
    self.secondDilatedConv =  tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1D(channels, 3, padding='same', dilation_rate=4,
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer()))
    self.fourthCBN = ConditionalBatchNorm(self.channels, self.isTraining)
    self.finalDilatedConv =  tfa.layers.SpectralNormalization(
              tf.keras.layers.Conv1D(channels, 3, padding='same', dilation_rate=8,
                    kernel_initializer=tf.initializers.Orthogonal,
                    kernel_regularizer=OrthogonalRegularizer()))
    

  def call(self, inputs, noise):
    outputs = self.firstCBN(inputs, noise)
    outputs = self.firstStack(outputs)
    outputs = self.secondCBN(outputs, noise)
    outputs = self.firstDilatedConv(outputs)
    residualOutputs = self.residualStack(inputs)
    outputs = outputs + residualOutputs
    outputs = self.thirdCBN(outputs, noise)
    outputs = self.secondDilatedConv(outputs)
    outputs = self.finalDilatedConv(outputs)
    return outputs


In [26]:
a = tf.random.normal((1, 256, 768))
z = tf.random.normal((1, 128, 1))
model = GeneratorBlock(768, False, 5)
model(a, z).shape

TensorShape([1, 512, 768])

## Conditional batch normalization + Relu

In [3]:
class ConditionalBatchNorm(tf.keras.Model):
  def __init__(self, units, isTraining, **kwargs):
    super(ConditionalBatchNorm, self).__init__(**kwargs)
    self.units = units
    self.isTraining = isTraining
    self.instanceNorm = tfa.layers.InstanceNormalization()
    self.mlp = tfa.layers.SpectralNormalization(
            tf.keras.layers.Dense(self.units,
                                  kernel_initializer=tf.keras.initializers.Orthogonal,
                                  kernel_regularizer=OrthogonalRegularizer()))
    self.randomIdx = np.random.randint(0, 128)
    self.relu = tf.keras.layers.ReLU()

  def call(self, inputs, noise):
    outputs = self.instanceNorm(inputs)
    deltaGamma = self.mlp(noise)[0][self.randomIdx]
    deltaBeta = self.mlp(noise)[0][self.randomIdx]
    outputs = tf.multiply(deltaGamma, outputs) + deltaBeta
    outputs = self.relu(outputs)
    return outputs

In [50]:
# input dim [1024, 256, 1]
input1 = tf.random.normal((1024, 256, 768))
noise = tf.random.normal((1024, 128, 1))
model = ConditionalBatchNorm(768, False)
output = model(input1, noise)
output.shape
# out dim [1024, 256, 128]

TensorShape([1024, 256, 768])

In [52]:
1//2

0.5